In [ ]:
from keras.models import Sequential, Model, model_from_json
import matplotlib.pyplot as plt
import keras 
import pickle
import wave
import os
import pandas as pd
import numpy as np
import sys
import warnings
import librosa
import librosa.display
import IPython.display as ipd
import tensorflow as tf
from sklearn.preprocessing import QuantileTransformer

In [ ]:
def speedNpitch(data):
    length_change = np.random.uniform(low=0.8, high = 1)
    speed_fac = 1.2  / length_change
    tmp = np.interp(np.arange(0,len(data),speed_fac),np.arange(0,len(data)),data)
    minlen = min(data.shape[0], tmp.shape[0])
    data *= 0
    data[0:minlen] = tmp[0:minlen]
    return data

In [ ]:
data, sampling_rate = librosa.load('../input/maleanger/male- anger-amp.wav')
ipd.Audio('../input/maleanger/male- anger-amp.wav')

In [ ]:
plt.figure(figsize=(15, 5))
librosa.display.waveplot(data, sr=sampling_rate)

# 1D CNN

In [ ]:
json_file = open('../input/1dicnn/model_json_out.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model_1D = model_from_json(loaded_model_json)
loaded_model_1D.load_weights('../input/1dicnn/1DCNN.h5')
print("Loaded model from disk")
opt = keras.optimizers.RMSprop(lr=0.00001, decay=1e-6)
loaded_model_1D.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
meandf = pd.read_csv("../input/normalization/mean.csv")
meandf = meandf.T
stddf = pd.read_csv("../input/normalization/std.csv")
stddf = stddf.T

In [ ]:
X, sample_rate = librosa.load('../input/maleanger/male- anger-amp.wav',res_type='kaiser_fast' ,duration=2.5,sr=44100,offset=1)

sample_rate = np.array(sample_rate)
mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13),axis=0)
newdf = pd.DataFrame(data=mfccs).T
new = newdf.to_numpy()
mean = meandf.to_numpy()
std = stddf.to_numpy()
new1d1 = (new-mean)/std
#print(new1d1)

In [ ]:
new1d1 = np.expand_dims(new1d1, axis=2)
newpred1d = loaded_model_1D.predict(new1d1,batch_size=16,verbose=1)

newpred1d

In [ ]:
filename = '../input/1dicnn/labels13'
infile = open(filename,'rb')
lb = pickle.load(infile)
infile.close()

final1d = newpred1d.argmax(axis=1)
final1d = final1d.astype(int).flatten()
final1d = (lb.inverse_transform((final1d)))
print(final1d)

# 2D CNN

In [ ]:
json_file = open('../input/2dicnn/model_json_2DCNN.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model_2D = model_from_json(loaded_model_json)
loaded_model_2D.load_weights('../input/2dicnn/best_model_2DCNN.h5')
print("Loaded model from disk")
opt2d = tf.keras.optimizers.Adam(lr=0.00017582232810420384)
loaded_model_2D.compile(loss='categorical_crossentropy', optimizer=opt2d, metrics=['accuracy'])

In [ ]:
wav, sr = librosa.core.load('../input/maleanger/male- anger-amp.wav', sr=44100, offset=0.5, duration=2.5, res_type='kaiser_fast')
input_length = 44100 * 2.5
df2d=np.empty(shape=(1, 30, 216))
if len(wav) > input_length:
    max_offset = len(wav) - input_length
    offset = np.random.randint(max_offset)
    wav = wav[offset:(input_length+offset)]
else:
    if input_length > len(wav):
        max_offset = input_length - len(wav)
        offset = np.random.randint(max_offset)
    else:
        offset = 0
    wav = np.pad(wav, (offset, int(input_length) - len(wav) - offset), "constant")
#wav = speedNpitch(wav)
MFCC = librosa.feature.mfcc(y=wav, sr=sr, n_mfcc=30)
df2d[0,0:30,0:216] = MFCC

In [ ]:
df2d = np.expand_dims(df2d, axis=3)
pred2d = loaded_model_2D.predict(df2d,batch_size=16,verbose=1)
pred2d

In [ ]:
filename = '../input/2dicnn/labels'
infile = open(filename,'rb')
lb = pickle.load(infile)
infile.close()
final2d = pred2d.argmax(axis=1)
final2d = final2d.astype(int).flatten()
final2d = (lb.inverse_transform((final2d)))
print(final2d)